<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/Bi-LSTM-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='tanh', recurrent_activation='sigmoid', go_backwards=True), input_shape=(len(test.columns)-1, 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 200)              81600     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 81,801
Trainable params: 81,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

14922/15625 [===========================>..] - ETA: 6s - loss: 0.0129 - accuracy: 0.9959

In [ ]:
test_y = test.pop('Label')
test_x = test

In [ ]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [ ]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

In [ ]:
confusion_matrix(test_y, pred_y)

In [ ]:
report = classification_report(test_y, pred_y, digits=5)
print(report)